Program analyzes short term stock movement based on new financial analyst ratings. Collects ratings and other relevant  from the Twitter API, and then draws price movements from Robinhood API. Random Forest to be fit when enough data is stored. Stores dataframes in Google's cloud.

Side Note: I realize this probably isn't a reliable way to invest, mostly did this project to showcase some of my data science skills.

In [ ]:
pip install tweepy --upgrade

In [ ]:
pip install schedule

In [ ]:
pip install robin_stocks

     |████████████████████████████████| 131 kB 7.4 MB/s 
     |████████████████████████████████| 3.6 MB 42.0 MB/s 


In [ ]:
from google.colab import auth
auth.authenticate_user()

project_id = 'sacred-particle-338100'
!gcloud config set project {project_id}

Updated property [core/project].


In [ ]:
import tweepy
from datetime import date
import robin_stocks as rs
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error as mae
import schedule
import time

In [ ]:
auth = tweepy.OAuthHandler('BryZGraJeHDlBZZgPcp4mbsb6','s16DhQCrBjoNCAn80QsdisocolHnAF6oRscc51YEkoDYFRf3Sz')
auth.set_access_token('1476313388238127106-RABw3vq8EnFH71J48Xm8XAckxS4fut', 'jwPvKUjbkRml9NvSK9ayz2mNHkLAABrzblItniaClVAQO')
api = tweepy.API(auth)
client = tweepy.Client(bearer_token = 'AAAAAAAAAAAAAAAAAAAAACw9XgEAAAAAVA5932TSYHD6TChvOaILsCq4a1c%3DMeL9LfR7kHjaa59STR5AHr0BmtsVtkotncOzAVxR3G6XSZWqtI')

In [ ]:
rs.robinhood.login(username = 'dontlook', password = 'dontlook', expiresIn = 72000, by_sms = True)

Collects tweets that include keywords suggesting news of analyst ratings sent during the morning period. Processes the data to collect the bank the analyst is from, the symbol of the company being rated, the initial rating, the new rating, the initial target, and current stock price information from Robinhood. Gathers info into a dataframe and sends frames into the cloud.

In [ ]:
def morningHair(client):
  today = date.today()
  usableToday = today.strftime('%Y-%m-%d')
  workinggroup1 = client.search_recent_tweets(query = 'maintains from:newsfilterio -is:retweet', end_time = usableToday + 'T14:20:00+00:00', start_time = usableToday + 'T05:00:00+00:00', max_results = 100)
  workinggroup2 = client.search_recent_tweets(query = 'downgrades from:newsfilterio -is:retweet', end_time = usableToday + 'T14:20:00+00:00', start_time = usableToday + 'T05:00:00+00:00', max_results = 100)
  workinggroup3 = client.search_recent_tweets(query = 'upgrades from:newsfilterio -is:retweet', end_time = usableToday + 'T14:20:00+00:00', start_time = usableToday + 'T05:00:00+00:00', max_results = 100)
  workinggroup4 = client.search_recent_tweets(query = 'initiates from:newsfilterio -is:retweet', end_time = usableToday + 'T14:20:00+00:00', start_time = usableToday + 'T05:00:00+00:00', max_results = 100)
  workinggroups = [workinggroup1, workinggroup2, workinggroup3, workinggroup4]
  initiatedFrame = pd.DataFrame(columns=['symbol','bank','init_rating','new_rating','new_target','current_price','closing_price'])
  vagueFrame = pd.DataFrame(columns = ['symbol','bank','init_rating','new_rating','current_price','closing_price'])
  updatedFrame = pd.DataFrame(columns = ['symbol','bank','init_rating','new_rating','init_target','new_target','current_price','closing_price'])
  for h in workinggroups:
    if(h != [None]):
      print('check')
    for i in h[0]:
      words = i.text.split(' ')
      if '' in words:
        words.remove('')
      symbol = words[0][1:]
      gang = rs.robinhood.stocks.get_quotes(inputSymbols = symbol)
      if gang != None and gang != [None] and len(words)>1:
        words = words[1:]
        bank = ''
        capsindex = 0
        caps = words[capsindex][0].isupper()
        if caps:
          bank += words[capsindex]
          capsindex += 1
          caps = words[capsindex][0].isupper()
        while caps:
          bank += ' ' + words[capsindex]
          capsindex += 1
          caps = words[capsindex][0].isupper()
        initialRating = ''
        newRating = ''
        if 'maintains' in words:
          if 'Buy' in words or 'Outperform' in words or 'Overweight' in words:
            initialRating = 'B'
            newRating = 'B'
          elif 'Neutral' in words or 'Equal-Weight' in words or 'Market Perform' in words or 'Hold' in words or 'Peer Perform':
            initialRating = 'H'
            newRating = 'H'
          elif 'Sell' in words or 'Underperform' in words or 'Underweight' in words:
            initialRating = 'S'
            newRating = 'S'
        elif 'initiates' in words:
          if 'Buy' in words or 'Outperform' in words or 'Overweight' in words:
            initialRating = 'N'
            newRating = 'B'
          elif 'Neutral' in words or 'Equal-Weight' in words or 'Market Perform' in words or 'Hold' in words or 'Peer Perform':
            initialRating = 'N'
            newRating = 'H'
          elif 'Sell' in words or 'Underperform' in words or 'Underweight' in words:
            initialRating = 'N'
            newRating = 'S'
        elif 'downgrades' in words:
          if 'Sell' in words or 'Underperform' in words or 'Underweight' in words:
            if 'Neutral' in words or 'Equal-Weight' in words or 'Market Perform' in words or 'Hold' in words or 'Peer Perform':
              initialRating = 'H'
              newRating = 'S'
            elif 'Buy' in words or 'Outperform' in words or 'Overweight' in words:
              initialRating = 'B'
              newRating = 'S'
          elif 'Neutral' in words or 'Equal-Weight' in words or 'Market Perform' in words or 'Hold' in words or 'Peer Perform':
            if 'Buy' in words or 'Outperform' in words or 'Overweight' in words:
              initialRating = 'B'
              newRating = 'H'
        elif 'upgrades' in words:
          if 'Buy' in words or 'Outperform' in words or 'Overweight' in words:
            if 'Neutral' in words or 'Equal-Weight' in words or 'Market Perform' in words or 'Hold' in words or 'Peer Perform':
              initialRating = 'H'
              newRating = 'B'
            elif 'Sell' in words or 'Underperform' in words or 'Underweight' in words:
              initialRating = 'S'
              newRating = 'B'
          elif 'Neutral' in words or 'Equal-Weight' in words or 'Market Perform' in words or 'Hold' in words or 'Peer Perform':
            if 'Sell' in words or 'Underperform' in words or 'Underweight' in words:
              initialRating = 'S'
              newRating = 'H'
        initialTarget = 0
        newTarget = 0
        key = 0
        targets = [x for x in words if x.startswith('$')]
        for j in range(len(targets)):
          if targets[j].startswith('$'):
            targets[j] = targets[j][1:]
          if 'h' in targets[j]:
            targets[j] = targets[j][:targets[j].index('h')]
          if targets[j].endswith('.'):
            targets[j] = targets[j][:-1]
        if 'initiates' in words and len(targets) == 1:
          key = 3
          newTarget = float(targets[0])
        elif len(targets) == 2:
          key = 1
          initialTarget = float(targets[0])
          newTarget = float(targets[1])
        elif len(targets) == 0:
          key = 2
        if gang[0]['last_extended_hours_trade_price'] == None or gang[0]['last_extended_hours_trade_price'] == [None]:
          currentPrice = float(gang[0]['last_trade_price'])
          closingPrice = float(gang[0]['last_trade_price'])
        else:
          currentPrice = float(gang[0]['last_extended_hours_trade_price'])
          closingPrice = float(gang[0]['last_trade_price'])
        if key == 1:
          updatedDict = {'symbol': symbol, 'bank': bank, 'init_rating': initialRating, 'new_rating': newRating, 'init_target': initialTarget, 'new_target': newTarget, 'current_price': currentPrice, 'closing_price': closingPrice}
          updatedFrame = updatedFrame.append(updatedDict, ignore_index = True)
        elif key == 2:
          vagueDict = {'symbol': symbol, 'bank': bank, 'init_rating': initialRating, 'new_rating': newRating, 'current_price': currentPrice, 'closing_price': closingPrice}
          vagueFrame = vagueFrame.append(vagueDict, ignore_index = True)
        elif key == 3:
          initiatedDict = {'symbol': symbol, 'bank': bank, 'init_rating': initialRating, 'new_rating': newRating, 'new_target': newTarget, 'current_price': currentPrice, 'closing_price': closingPrice}
          initiatedFrame = initiatedFrame.append(initiatedDict, ignore_index = True)
  updatedFrame['target_ratio'] = updatedFrame.new_target / updatedFrame.init_target
  updatedFrame['TtoP_ratio'] = updatedFrame.new_target / updatedFrame.current_price
  initiatedFrame['TtoP_ratio'] = initiatedFrame.new_target / initiatedFrame.current_price
  updatedcsv = updatedFrame.to_csv(path_or_buf = '/tmp/updatedFrameinTraining.csv')
  !gsutil cp '/tmp/updatedFrameinTraining.csv' gs://{'tweetrobintweet'}/
  initiatedcsv = initiatedFrame.to_csv(path_or_buf = '/tmp/initiatedFrameinTraining.csv')
  !gsutil cp '/tmp/initiatedFrameinTraining.csv' gs://{'tweetrobintweet'}/
  vaguecsv = vagueFrame.to_csv(path_or_buf = '/tmp/vagueFrameinTraining.csv')
  !gsutil cp '/tmp/vagueFrameinTraining.csv' gs://{'tweetrobintweet'}/

Method adding the stock price change to the dataframe the day after the rating was announced.

In [ ]:
def yAdder(dataframe):
  symbols = []
  for i, row in dataframe.iterrows():
    ticker = row.symbol
    oldPrice = row.current_price
    gang = rs.robinhood.stocks.get_quotes(ticker)
    if gang[0] != [None] and gang[0] != None:
      if gang[0]['last_extended_hours_trade_price'] == None or gang[0]['last_extended_hours_trade_price'] == [None]:
        changePrice = gang[0]['last_trade_price']
      else:
        changePrice = gang[0]['last_extended_hours_trade_price']
      symbols.append(float(changePrice)/float(oldPrice))
    else:
      symbols.append(0)
  dataframe['Y'] = symbols

Adds the price changes to the unfinished dataframes, appends the newly finished dataframe to the big dataframe that the models are trained on, sends those models up to the cloud.

In [ ]:
def morningSickness():
  !gsutil cp gs://{'tweetrobintweet'}/updatedFrameinTraining.csv /tmp/updatedFrameinTraining.csv
  downloadeduframe = pd.read_csv('/tmp/updatedFrameinTraining.csv', index_col=0)
  !gsutil cp gs://{'tweetrobintweet'}/initiatedFrameinTraining.csv /tmp/initiatedFrameinTraining.csv
  downloadediframe = pd.read_csv('/tmp/initiatedFrameinTraining.csv', index_col = 0)
  !gsutil cp gs://{'tweetrobintweet'}/vagueFrameinTraining.csv /tmp/vagueFrameinTraining.csv
  downloadedvframe = pd.read_csv('/tmp/vagueFrameinTraining.csv', index_col = 0)
  yAdder(downloadeduframe)
  print(downloadeduframe.shape)
  yAdder(downloadediframe)
  yAdder(downloadedvframe)
  !gsutil cp gs://{'tweetrobintweet'}/updatedFrame.csv /tmp/updatedFrame.csv
  ucsv = pd.read_csv('/tmp/updatedFrame.csv', index_col=0)
  print(ucsv.shape)
  !gsutil cp gs://{'tweetrobintweet'}/initiatedFrame.csv /tmp/initiatedFrame.csv
  icsv = pd.read_csv('/tmp/initiatedFrame.csv', index_col = 0)
  !gsutil cp gs://{'tweetrobintweet'}/vagueFrame.csv /tmp/vagueFrame.csv
  vcsv = pd.read_csv('/tmp/vagueFrame.csv', index_col = 0)
  udf = ucsv.append(downloadeduframe)
  idf = icsv.append(downloadediframe)
  vdf = vcsv.append(downloadedvframe)
  utocloud = udf.to_csv(path_or_buf = '/tmp/updatedFrame.csv')
  print(udf.shape)
  !gsutil cp '/tmp/updatedFrame.csv' gs://{'tweetrobintweet'}/
  itocloud = idf.to_csv(path_or_buf = '/tmp/initiatedFrame.csv')
  !gsutil cp '/tmp/initiatedFrame.csv' gs://{'tweetrobintweet'}/
  vtocloud = vdf.to_csv(path_or_buf = '/tmp/vagueFrame.csv')
  !gsutil cp '/tmp/vagueFrame.csv' gs://{'tweetrobintweet'}/

Schedules the running of the program.

In [ ]:
schedule.every().day.at('14:22').do(morningSickness)
schedule.every().day.at('14:25').do(morningHair, client = client)

while True:
  schedule.run_pending()
  time.sleep(1)

Copying gs://tweetrobintweet/updatedFrameinTraining.csv...
/ [1 files][  8.6 KiB/  8.6 KiB]                                                
Operation completed over 1 objects/8.6 KiB.                                      
Copying gs://tweetrobintweet/initiatedFrameinTraining.csv...
/ [1 files][  730.0 B/  730.0 B]                                                
Operation completed over 1 objects/730.0 B.                                      
Copying gs://tweetrobintweet/vagueFrameinTraining.csv...
/ [1 files][  346.0 B/  346.0 B]                                                
Operation completed over 1 objects/346.0 B.                                      
(108, 11)
Copying gs://tweetrobintweet/updatedFrame.csv...
/ [1 files][102.6 KiB/102.6 KiB]                                                
Operation completed over 1 objects/102.6 KiB.                                    
(1048, 11)
Copying gs://tweetrobintweet/initiatedFrame.csv...
/ [1 files][ 14.8 KiB/ 14.8 KiB]                  

TypeError: ignored

Model training will be done once more data is collected. Below is a basic framework for the type of model being pursued.

In [ ]:
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.linear_model import LinearRegression
import numpy as np

In [ ]:
!gsutil cp gs://{'tweetrobintweet'}/updatedFrame.csv /tmp/updatedFrame.csv
ucsv = pd.read_csv('/tmp/updatedFrame.csv', index_col=0)
print(ucsv.shape)
ucsv['preHours'] = ucsv.current_price/ucsv.closing_price
ucsv.dropna(inplace = True)
ucsvf = ucsv[ucsv['Y'] != 0]
print(ucsvf.shape)


Copying gs://tweetrobintweet/updatedFrame.csv...
/ [1 files][102.6 KiB/102.6 KiB]                                                
Operation completed over 1 objects/102.6 KiB.                                    
(1048, 11)
(1043, 12)


In [ ]:
from xgboost import XGBRegressor

In [ ]:
y = ucsvf.Y
X = ucsvf.drop('Y', axis = 1)
X = ucsvf[['bank','new_rating', 'init_rating', 'target_ratio','TtoP_ratio','preHours', 'current_price']]
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=585)
hothothot = OneHotEncoder(handle_unknown='ignore')
ordinary = OrdinalEncoder(categories = [['S','H','B'],['S','H','B']])
transformer = ColumnTransformer([('step1', hothothot, ['bank']), ('step2', ordinary, ['init_rating', 'new_rating'])])
booster = XGBRegressor(n_estimators = 1000, learning_rate = .01, max_depth = 7, random_state = 585)
xpipeline = Pipeline(steps=[('megatron', transformer),('modelo', booster)])
transformer.fit_transform(X_train)
X_valid = transformer.transform(X_test)
fit_params = {'modelo__eval_set' : [(X_valid, y_test)], 'modelo__early_stopping_rounds':5}
xpipeline.fit(X_train, y_train, **fit_params)
letssee = xpipeline.predict(X_test)
score = mae(y_test, letssee)
print(score)

[23:32:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:0.500595
Will train until validation_0-rmse hasn't improved in 5 rounds.
[1]	validation_0-rmse:0.495682
[2]	validation_0-rmse:0.490818
[3]	validation_0-rmse:0.486004
[4]	validation_0-rmse:0.481238
[5]	validation_0-rmse:0.47652
[6]	validation_0-rmse:0.471851
[7]	validation_0-rmse:0.467228
[8]	validation_0-rmse:0.462653
[9]	validation_0-rmse:0.458124
[10]	validation_0-rmse:0.45364
[11]	validation_0-rmse:0.449203
[12]	validation_0-rmse:0.44481
[13]	validation_0-rmse:0.440462
[14]	validation_0-rmse:0.436159
[15]	validation_0-rmse:0.431899
[16]	validation_0-rmse:0.427682
[17]	validation_0-rmse:0.423508
[18]	validation_0-rmse:0.419377
[19]	validation_0-rmse:0.415287
[20]	validation_0-rmse:0.41124
[21]	validation_0-rmse:0.407233
[22]	validation_0-rmse:0.403267
[23]	validation_0-rmse:0.399342
[24]	validation_0-rmse:0.395457
[25]	validation_0-rmse:

In [ ]:
y = ucsvf.Y
X = ucsvf.drop('Y', axis = 1)
X = ucsvf[['bank','new_rating', 'init_rating', 'target_ratio','TtoP_ratio','preHours', 'current_price']]
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=585)
hothothot = OneHotEncoder(handle_unknown='ignore')
ordinary = OrdinalEncoder(categories = [['S','H','B'],['S','H','B']])
transformer = ColumnTransformer([('step1', hothothot, ['bank']), ('step2', ordinary, ['init_rating', 'new_rating'])])
forest = RandomForestRegressor(n_estimators = 506, max_depth = 7, random_state = 585)
#forest = RandomForestRegressor(random_state = 585)
pipeline = Pipeline(steps=[('megatron', transformer),('modelo', forest)])
#griddy = GridSearchCV(pipeline, {'modelo__n_estimators':[5,25,125,625], 'modelo__max_depth':[1,2,3,4,5]})
#grid_results = griddy.fit(X_train, y_train)
#print(grid_results.best_estimator_)
results = cross_val_score(pipeline, X_train, y_train, cv = 5, scoring = 'neg_mean_absolute_error')
print(np.mean(results))
pipeline.fit(X_train, y_train)
tryguys = pipeline.predict(X_test)
score = mae(y_test, tryguys)
print(score)

-0.035653431525095244
0.03870126745494122


In [ ]:
!gsutil cp gs://{'tweetrobintweet'}/updatedFrameinTraining.csv /tmp/updatedFrameinTraining.csv
realworld = pd.read_csv('/tmp/updatedFrameinTraining.csv', index_col=0)
realworld['preHours'] = realworld.current_price/realworld.closing_price
realworld.dropna(inplace = True)
realX = realworld[['bank','new_rating', 'init_rating', 'target_ratio','TtoP_ratio','preHours']]
fingerscrossed = xpipeline.predict(realX)
realworld['theoreticalY'] = fingerscrossed
realworld.sort_values(by = 'theoreticalY', axis = 0, inplace = True, ascending = False)
realworld.head(5)

Copying gs://tweetrobintweet/updatedFrameinTraining.csv...
/ [1 files][  8.6 KiB/  8.6 KiB]                                                
Operation completed over 1 objects/8.6 KiB.                                      


,symbol,bank,init_rating,new_rating,init_target,new_target,current_price,closing_price,target_ratio,TtoP_ratio,preHours,theoreticalY
73,KR,Deutsche Bank,H,H,45.0,43.0,45.0100,45.04,0.955556,0.955343,0.999334,1.015903
67,AXL,Deutsche Bank,H,H,11.0,10.0,8.1600,7.91,0.909091,1.225490,1.031606,1.015903
69,LIND,Deutsche Bank,H,H,17.0,16.0,16.7800,16.78,0.941176,0.953516,1.000000,1.015903
80,M,Deutsche Bank,H,H,32.0,34.0,24.4299,24.42,1.062500,1.391737,1.000405,1.015903
75,CARR,Deutsche Bank,H,H,53.0,54.0,45.6800,45.29,1.018868,1.182137,1.008611,1.015903
